In [ ]:
import matplotlib.pyplot as plt
import squarify
import matplotlib
import json
import operator
import os,sys
from nltk.corpus import stopwords
import re
import csv
import pandas as pd
import collections

In [ ]:
def read_file(path):
    data_list = []
    with open(path,'r') as f:
        for line in f:
            data_list.append(line[:-1])
    return data_list

def flatten(d, parent_key='', sep='.'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

def get_text(data):
    flattern_data = flatten(data)
    text = data['text']
    for key in flattern_data:
        if key.split('.')[-1] == 'full_text':
            text = flattern_data[key]
    return text  

In [ ]:
###########used to read a single json file, it will return a list of orginal tweet information
def read_json(file):
    data = []
    print('Read '+file)
    with open(file, 'r') as f:
        for index,line in enumerate(f):
            #if index > 30000:
            #    break
            try:
                if line.strip():
                    text = json.loads(line)
                    #text = ' '.join(get_word(text))
                    data.append(text)
            except Exception as e:
                print(e)
                print("can't read line "+str(index))
    return data


def read_json_dict(file):
    data = {}
    try:
        with open(file,'r') as f:
            for line in f:
                if line.strip():
                    data[line.split()[0]] = float(line.split()[1])
    except Exception as e:
        print(e)
        print("can't open file "+file)
    
    return data

def read_folder(path):
    text_data = []

    for index,filename in enumerate(sorted(os.listdir(path))):
        try:
            d = read_json(path+filename)
            text_data += d
        except Exception as e:
            print(e)
            print("Can't open file "+str(filename))

    return text_data

In [ ]:
def data_separate(data):
    label = []
    value = []
    occupation = data.split(' ')[0]
    for d in data.split(' ')[8:]:
        label.append(d.split(':')[0])
        #print(d.split(':'))
        value.append(float(d.split(':')[1][0:8]))
    return occupation, label, value

def draw_treemap(data_list):
    
    for index,data in enumerate(data_list[2:]):
        
        occupation, labels, value = data_separate(data)
        norm = matplotlib.colors.Normalize(vmin=min(value), vmax=max(value))
        colors = [matplotlib.cm.Blues(norm(v)) for v in value]
        print(float(occupation))
        print(str(index))
        print(labels)
        squarify.plot(sizes=value, label=labels, color = colors)#, alpha=.8)
        plt.axis('off')
        plt.show()
        
def clean(text):
    text = re.sub(r'[^\x00-\x7F]+','', text)
    text = str.replace(text,',',' ')
    text = str.replace(text,"'",' ')
    text = str.replace(text,'"',' ')
    text = str.replace(text,'!',' ')
    text = str.replace(text,'^',' ')
    text = str.replace(text,'(',' ')
    text = str.replace(text,')',' ')
    text = str.replace(text,'%',' ')
    text = str.replace(text,'-',' ')
    text = str.replace(text,'_',' ')
    text = str.replace(text,'|',' ')
    text = str.replace(text,'.',' ')
    text = str.replace(text,':',' ')
    #text = str.replace(text,'@',' ')
    #text = str.replace(text,'#','')
    text = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",text).split())

    text = re.sub('\s+', ' ',text).strip()
    text = text.split(' ')
    new_text = []
    for each in text:
        if(str.find(each,'http') != -1):
            continue
        if not each.isalnum():
            continue
        new_text.append(str.lower(each));
    text = ' '.join(new_text)

    return text

def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

def get_word(text):
    special_word = ['rt','co','amp'] #here is the word we don't want it to show up in the sentence
    words = []
    for w in clean(text).split():
        if isEnglish(w):
            if w not in set(stopwords.words('english')) and w not in special_word:
                words.append(w)
    return words

In [ ]:
####file path is the filename to store the BTM topic result
file_path = ''
data_list = read_file(file_path)
draw_treemap(data_list)